# 🚀 Qwen2-VL vs Qwen3-VL: Complete Architecture Deep Dive

This notebook provides comprehensive coverage of:
1. **Architecture Deep Dive** - Vision encoder, position encoding, activation functions
2. **GPU-Optimized Configurations** - T4, A100, H100, B200 specific settings
3. **DeepStack & EVS** - Qwen3-VL exclusive features explained
4. **Production-Ready Inference** - Optimized code for each GPU tier

---

## Executive Summary

```
╔═══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                              QWEN2-VL vs QWEN3-VL ARCHITECTURE EVOLUTION                          ║
╠═══════════════════════════════════════════════════════════════════════════════════════════════════╣
║                                                                                                   ║
║  QWEN2-VL (2024)                                    QWEN3-VL (2025)                               ║
║  ══════════════                                     ══════════════                                ║
║                                                                                                   ║
║  Image/Video ──▶ Conv3D (NO BIAS)                   Image/Video ──▶ Conv3D (WITH BIAS)            ║
║        │                                                  │                                       ║
║        ▼                                                  ▼                                       ║
║  3D RoPE Position Encoding                          Learned + Interpolated Position Embed         ║
║        │                                                  │                                       ║
║        ▼                                                  ▼                                       ║
║  Vision Transformer Blocks                          Vision Transformer Blocks ──┐                 ║
║  • LayerNorm                                        • LayerNorm                 │ DeepStack       ║
║  • Attention (QKV + RoPE)                           • Attention (QKV + RoPE)    │ Mergers at      ║
║  • MLP (QuickGELU, WITH bias)                       • MLP (SiLU, NO bias)       │ intermediate    ║
║        │                                                  │                     │ layers          ║
║        ▼                                                  ▼                     │                 ║
║  Single Patch Merger                                Main Merger + DeepStack ◄───┘                 ║
║        │                                                  │                                       ║
║        ▼                                                  ▼                                       ║
║  Qwen2 LLM Backbone                                 Qwen3 LLM with DeepStack Injection            ║
║  (visual tokens at embed level)                     (multi-scale features in early layers)        ║
║                                                                                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════════╝
```

## Key Architectural Differences Explained

### 1. Patch Embedding (Conv3D)
- **Qwen2-VL**: `bias=False` - Relies on LayerNorm for centering
- **Qwen3-VL**: `bias=True` - Learnable offset captures low-frequency patterns

### 2. Position Encoding
- **Qwen2-VL**: Pure 3D RoPE from grid coordinates
- **Qwen3-VL**: Learned embeddings + bilinear interpolation for any resolution + partial RoPE

### 3. MLP Activation
- **Qwen2-VL**: QuickGELU `x * σ(1.702x)` - Fast approximation
- **Qwen3-VL**: SiLU `x * σ(x)` - Smoother gradients, modern standard

### 4. DeepStack (Qwen3-VL Only)
- Extracts features from intermediate vision layers
- Injects multi-scale features into early LLM layers
- Better fine-grained visual understanding

### 5. EVS - Efficient Video Sampling (Qwen3-VL Only)
- Content-aware token pruning based on frame similarity
- Reduces video tokens by 30-70% with minimal quality loss
- Enables processing of much longer videos


## GPU Architecture Impact on Qwen VL Models

### Why GPU Architecture Matters

Each GPU architecture has specific capabilities that affect how Qwen VL models run:

| GPU | VRAM | Bandwidth | Key Features | Best For |
|-----|------|-----------|--------------|----------|
| **T4** | 16GB | 320 GB/s | FP16, SDPA only | Qwen2-VL-2B, Qwen3-VL-4B (4-bit) |
| **A100** | 40/80GB | 2.0 TB/s | BF16, FlashAttn v2 | Qwen2/3-VL 7B/8B full precision |
| **H100** | 80GB | 3.35 TB/s | FP8, FlashAttn v3 | All models + FP8 quantization |
| **B200** | 192GB | 8.0 TB/s | FP4 (future), 128K ctx | Qwen2-VL-72B single GPU! |

### Attention Backend Selection

```
┌─────────────────────────────────────────────────────────────────────────────┐
│  Backend         │ T4 (SM 7.5) │ A100 (SM 8.0) │ H100 (SM 9.0) │ B200      │
├─────────────────────────────────────────────────────────────────────────────┤
│  FLASH_ATTN      │ ❌          │ ✅            │ ✅ (v3)       │ ✅        │
│  TORCH_SDPA      │ ✅ (only)   │ ✅            │ ✅            │ ✅        │
│  XFORMERS        │ ⚠️ Qwen2    │ ⚠️ Qwen2      │ ⚠️ Qwen2      │ ⚠️ Qwen2  │
└─────────────────────────────────────────────────────────────────────────────┘
Note: Qwen3-VL only supports FLASH_ATTN, TORCH_SDPA, and ROCM_AITER_FA
```

### Memory Budget Examples

**T4 (16GB) with Qwen2-VL-2B:**
- Model weights: ~5 GB
- KV Cache (2K ctx): ~2 GB  
- Activations: ~1 GB
- CUDA context: ~1.5 GB
- Vision encoder temp: ~2 GB
- **Available for batch: ~4.5 GB**

**H100 (80GB) with Qwen3-VL-8B:**
- Model weights: ~17 GB
- KV Cache (32K ctx): ~25 GB
- Activations: ~8 GB
- DeepStack buffers: ~2 GB
- Prefix cache: ~15 GB
- **Available for batch: ~13 GB**


In [ ]:
# Cell 1: GPU Detection & Environment Setup
import subprocess
import sys

print("=" * 80)
print("🔍 GPU DETECTION & ENVIRONMENT SETUP")
print("=" * 80)

# Check GPU
!nvidia-smi --query-gpu=name,memory.total,compute_cap --format=csv

import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    cc = torch.cuda.get_device_capability()
    
    print(f"\n✅ GPU: {gpu_name}")
    print(f"✅ Memory: {gpu_memory:.1f} GB")
    print(f"✅ Compute Capability: SM {cc[0]}.{cc[1]}")
    
    # Determine architecture and recommendations
    if cc[0] >= 10:
        arch = "Blackwell (B200)"
        recommended = "Qwen3-VL-8B (128K context) or Qwen2-VL-72B"
    elif cc[0] >= 9:
        arch = "Hopper (H100)"
        recommended = "Qwen3-VL-8B or Qwen3-VL-30B-A3B (MoE)"
    elif cc[0] >= 8:
        arch = "Ampere (A100)"
        recommended = "Qwen3-VL-8B or Qwen2-VL-7B"
    else:
        arch = "Turing (T4)"
        recommended = "Qwen2-VL-2B or Qwen3-VL-4B (4-bit)"
    
    print(f"✅ Architecture: {arch}")
    print(f"✅ Recommended: {recommended}")
    
    GPU_INFO = {"name": gpu_name, "memory": gpu_memory, "sm": cc[0]}
else:
    print("❌ No GPU available!")
    sys.exit(1)

print("\n📦 Installing dependencies...")
%pip install -q vllm>=0.6.0 pillow requests numpy
print("✅ Dependencies installed!")


In [ ]:
# Cell 2: GPU-Optimized Configuration Selection with Detailed Documentation

def get_optimal_config(sm_version: int, vram_gb: float, prefer_qwen3: bool = True):
    """
    Get optimal vLLM configuration based on GPU architecture.
    
    Architecture-Specific Optimizations:
    ====================================
    
    T4 (SM 7.5, 16GB GDDR6, 320 GB/s):
    - No FlashAttention v2 (requires SM 8.0+) → Uses TORCH_SDPA
    - No BF16 support → Must use FP16 (dtype="half")
    - No FP8 quantization → Use bitsandbytes INT4 for larger models
    - Limited memory → enforce_eager=True saves ~500MB CUDA graphs
    - Low bandwidth → Reduce max_pixels to minimize vision processing
    
    A100 (SM 8.0, 80GB HBM2e, 2.0 TB/s):
    - FlashAttention v2 supported → Significant speedup
    - BF16 native support → Better numerical stability
    - 6x T4 bandwidth → Can handle larger images/more batching
    - Use prefix caching for repeated prompts
    
    H100 (SM 9.0, 80GB HBM3, 3.35 TB/s):
    - FlashAttention v3 (Hopper-optimized)
    - FP8 hardware support → 2x throughput when quantized
    - 10x T4 bandwidth → Ultra-fast vision processing
    - Transformer Engine for automatic mixed precision
    
    B200 (SM 10.0, 192GB HBM3e, 8.0 TB/s):
    - FP4 hardware support (future, ~4x throughput)
    - 25x T4 bandwidth → Massive throughput
    - Huge memory → 128K context, 4K images, or 72B models
    
    Args:
        sm_version: CUDA Compute Capability major version (7=Turing, 8=Ampere, etc.)
        vram_gb: GPU memory in GB
        prefer_qwen3: Whether to prefer Qwen3-VL over Qwen2-VL
        
    Returns:
        dict: vLLM configuration optimized for the GPU
    """
    
    # ══════════════════════════════════════════════════════════════════════════════
    # T4 / TURING (SM 7.5, 16GB)
    # ══════════════════════════════════════════════════════════════════════════════
    if sm_version < 8:
        # Architecture constraints:
        # - No FlashAttention v2 (falls back to TORCH_SDPA for vision encoder)
        # - No BF16 tensor cores (must use FP16)
        # - Limited to ~14GB usable (CUDA context ~1.5GB)
        
        if prefer_qwen3:
            return {
                "model": "Qwen/Qwen3-VL-4B-Instruct",
                "trust_remote_code": True,
                "dtype": "half",  # T4 doesn't support BF16
                
                # Quantization: Required for 4B model on 16GB
                "quantization": "bitsandbytes",
                "load_format": "bitsandbytes",
                
                # Memory optimization
                "gpu_memory_utilization": 0.92,  # Leave some headroom
                "max_model_len": 2048,  # Limited context for memory
                "enforce_eager": True,  # Save ~500MB by disabling CUDA graphs
                
                # Batch size limited due to memory
                "max_num_seqs": 4,
                
                # Media limits - conservative for T4
                "limit_mm_per_prompt": {"image": 1, "video": 1},
                "mm_processor_kwargs": {
                    "min_pixels": 784,  # 28×28 minimum
                    "max_pixels": 512000,  # ~720×720 max
                    "video_pruning_rate": 0.5,  # EVS: Prune 50% for Qwen3-VL
                },
                
                # Disable advanced features that need memory
                "enable_prefix_caching": False,
                "enable_chunked_prefill": False,
            }
        else:
            return {
                "model": "Qwen/Qwen2-VL-2B-Instruct",
                "trust_remote_code": True,
                "dtype": "half",  # FP16 only
                
                # No quantization needed for 2B
                "gpu_memory_utilization": 0.90,
                "max_model_len": 2048,
                "enforce_eager": True,  # Save memory
                "max_num_seqs": 4,
                
                # Qwen2-VL: No video support on T4 (too slow)
                "limit_mm_per_prompt": {"image": 1, "video": 0},
                "mm_processor_kwargs": {
                    "min_pixels": 784,
                    "max_pixels": 512000,
                },
                
                "enable_prefix_caching": False,
                "enable_chunked_prefill": False,
            }
    
    # ══════════════════════════════════════════════════════════════════════════════
    # A100 / AMPERE (SM 8.0, 40-80GB)
    # ══════════════════════════════════════════════════════════════════════════════
    elif sm_version < 9:
        # Architecture features:
        # - FlashAttention v2 supported (significant speedup)
        # - BF16 native tensor core support
        # - 2.0 TB/s bandwidth (6x T4)
        # - Up to 80GB HBM2e memory
        
        if prefer_qwen3:
            return {
                "model": "Qwen/Qwen3-VL-8B-Instruct",
                "trust_remote_code": True,
                "dtype": "bfloat16",  # Use BF16 for better stability
                
                # Leverage abundant memory
                "gpu_memory_utilization": 0.95,
                "max_model_len": 16384,  # 16K context
                "enforce_eager": False,  # Enable CUDA graphs
                "max_num_seqs": 32,  # Good concurrency
                
                # Can handle multiple images/videos
                "limit_mm_per_prompt": {"image": 8, "video": 4},
                "mm_processor_kwargs": {
                    "min_pixels": 784,
                    "max_pixels": 1572864,  # ~1280×1280
                    "video_pruning_rate": 0.3,  # EVS: 30% pruning
                },
                
                # Enable advanced features
                "enable_prefix_caching": True,  # Leverage memory
                "enable_chunked_prefill": True,  # Better long-sequence handling
            }
        else:
            return {
                "model": "Qwen/Qwen2-VL-7B-Instruct",
                "trust_remote_code": True,
                "dtype": "bfloat16",
                "gpu_memory_utilization": 0.95,
                "max_model_len": 16384,
                "enforce_eager": False,
                "max_num_seqs": 32,
                "limit_mm_per_prompt": {"image": 8, "video": 2},  # Less video (no EVS)
                "mm_processor_kwargs": {
                    "min_pixels": 784,
                    "max_pixels": 1572864,
                },
                "enable_prefix_caching": True,
                "enable_chunked_prefill": True,
            }
    
    # ══════════════════════════════════════════════════════════════════════════════
    # H100 / HOPPER (SM 9.0, 80GB HBM3)
    # ══════════════════════════════════════════════════════════════════════════════
    elif sm_version < 10:
        # Architecture features:
        # - FlashAttention v3 (Hopper-optimized kernels)
        # - FP8 hardware support (2x throughput)
        # - 3.35 TB/s bandwidth (10x T4!)
        # - Transformer Engine for automatic mixed precision
        
        if prefer_qwen3:
            return {
                "model": "Qwen/Qwen3-VL-8B-Instruct",
                "trust_remote_code": True,
                "dtype": "bfloat16",
                
                # H100 can handle 32K context easily
                "gpu_memory_utilization": 0.95,
                "max_model_len": 32768,  # 32K context
                "enforce_eager": False,
                "max_num_seqs": 64,  # High concurrency
                
                # Handle many images/videos
                "limit_mm_per_prompt": {"image": 16, "video": 8},
                "mm_processor_kwargs": {
                    "min_pixels": 784,
                    "max_pixels": 2073600,  # 1920×1080 (1080p)
                    "video_pruning_rate": 0.3,  # EVS
                },
                
                "enable_prefix_caching": True,
                "enable_chunked_prefill": True,
                
                # Optional: Add for maximum throughput
                # "quantization": "fp8",  # 2x throughput
                # "kv_cache_dtype": "fp8",  # 2x KV capacity
            }
        else:
            return {
                "model": "Qwen/Qwen2-VL-7B-Instruct",
                "trust_remote_code": True,
                "dtype": "bfloat16",
                "gpu_memory_utilization": 0.95,
                "max_model_len": 32768,
                "enforce_eager": False,
                "max_num_seqs": 64,
                "limit_mm_per_prompt": {"image": 16, "video": 4},
                "mm_processor_kwargs": {
                    "min_pixels": 784,
                    "max_pixels": 2073600,
                },
                "enable_prefix_caching": True,
                "enable_chunked_prefill": True,
            }
    
    # ══════════════════════════════════════════════════════════════════════════════
    # B200 / BLACKWELL (SM 10.0, 192GB HBM3e)
    # ══════════════════════════════════════════════════════════════════════════════
    else:
        # Architecture features:
        # - FP4 hardware support (future, ~4x throughput)
        # - 8.0 TB/s bandwidth (25x T4!)
        # - 192GB HBM3e memory
        # - 2nd Gen Transformer Engine
        # - Can run Qwen2-VL-72B on a SINGLE GPU!
        
        if prefer_qwen3:
            return {
                "model": "Qwen/Qwen3-VL-8B-Instruct",
                "trust_remote_code": True,
                "dtype": "bfloat16",
                
                # Ultra-long context possible!
                "gpu_memory_utilization": 0.95,
                "max_model_len": 131072,  # 128K context!
                "enforce_eager": False,
                "max_num_seqs": 128,  # Massive batch
                
                # Handle many high-res images/videos
                "limit_mm_per_prompt": {"image": 32, "video": 16},
                "mm_processor_kwargs": {
                    "min_pixels": 784,
                    "max_pixels": 4147200,  # 4K resolution!
                    "video_pruning_rate": 0.3,  # EVS
                },
                
                "enable_prefix_caching": True,
                "enable_chunked_prefill": True,
            }
        else:
            return {
                "model": "Qwen/Qwen2-VL-72B-Instruct",  # FULL 72B on single GPU!
                "trust_remote_code": True,
                "dtype": "bfloat16",
                "gpu_memory_utilization": 0.95,
                "max_model_len": 32768,  # 32K with 72B model
                "enforce_eager": False,
                "max_num_seqs": 32,
                "limit_mm_per_prompt": {"image": 16, "video": 8},
                "mm_processor_kwargs": {
                    "min_pixels": 784,
                    "max_pixels": 4147200,  # 4K
                },
                "enable_prefix_caching": True,
                "enable_chunked_prefill": True,
            }

# ════════════════════════════════════════════════════════════════════════════════════
# Configuration Selection
# ════════════════════════════════════════════════════════════════════════════════════

# Set PREFER_QWEN3 = False to use Qwen2-VL instead
PREFER_QWEN3 = True

config = get_optimal_config(GPU_INFO["sm"], GPU_INFO["memory"], prefer_qwen3=PREFER_QWEN3)

print("⚡ SELECTED CONFIGURATION")
print("=" * 80)
print(f"   GPU: {GPU_INFO['name']} ({GPU_INFO['memory']:.1f} GB, SM {GPU_INFO['sm']})")
print("=" * 80)
for k, v in config.items():
    print(f"   {k}: {v}")
print("=" * 80)

# Display configuration rationale
if GPU_INFO['sm'] < 8:
    print("\n📝 T4 Optimizations Applied:")
    print("   • Using TORCH_SDPA (no FlashAttention v2 on SM 7.x)")
    print("   • FP16 dtype (no BF16 tensor cores)")
    print("   • enforce_eager=True (saves ~500MB CUDA memory)")
    print("   • max_pixels=512000 (~720×720 to fit in memory)")
    if PREFER_QWEN3:
        print("   • bitsandbytes INT4 quantization for Qwen3-VL-4B")
        print("   • EVS video_pruning_rate=0.5 (50% token reduction)")
elif GPU_INFO['sm'] < 9:
    print("\n📝 A100 Optimizations Applied:")
    print("   • FlashAttention v2 enabled (SM 8.0)")
    print("   • BF16 dtype (tensor core optimized)")
    print("   • Prefix caching enabled (leverage 80GB memory)")
    if PREFER_QWEN3:
        print("   • EVS video_pruning_rate=0.3 (30% token reduction)")
elif GPU_INFO['sm'] < 10:
    print("\n📝 H100 Optimizations Applied:")
    print("   • FlashAttention v3 (Hopper-optimized)")
    print("   • 32K context enabled")
    print("   • Consider FP8 quantization for 2x throughput")
    if PREFER_QWEN3:
        print("   • EVS video_pruning_rate=0.3 (30% token reduction)")
else:
    print("\n📝 B200 Optimizations Applied:")
    print("   • 128K context enabled (with Qwen3-VL)")
    print("   • 4K image resolution supported")
    print("   • Massive batch size (128 sequences)")
    if not PREFER_QWEN3:
        print("   • Running Qwen2-VL-72B on SINGLE GPU!")


In [ ]:
# Cell 3: Initialize vLLM Engine
from vllm import LLM, SamplingParams
import time

print("⚡ Initializing vLLM engine...")
print(f"   Model: {config['model']}")

init_start = time.time()
llm = LLM(**config)
init_time = time.time() - init_start

print(f"\n✅ Engine initialized in {init_time:.1f}s")

allocated = torch.cuda.memory_allocated() / (1024**3)
reserved = torch.cuda.memory_reserved() / (1024**3)
print(f"\n📊 GPU Memory: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved")


In [ ]:
# Cell 4: Create Test Image
from PIL import Image, ImageDraw

def create_test_image(width=1024, height=1024):
    """Create test dashboard image."""
    img = Image.new('RGB', (width, height), '#0f1420')
    draw = ImageDraw.Draw(img)
    
    # Top bar
    draw.rectangle([0, 0, width, 60], fill='#1a2535')
    draw.text((20, 20), "Dashboard - Analytics", fill='#ffffff')
    
    # Sidebar
    draw.rectangle([0, 60, 200, height], fill='#141d2b')
    items = ["Dashboard", "Analytics", "Reports", "Settings", "Users"]
    for i, item in enumerate(items):
        y = 100 + i * 50
        draw.rectangle([10, y, 190, y + 40], fill='#1f2d42')
        draw.text((30, y + 10), item, fill='#b0b0b0')
    
    # Cards
    for row in range(2):
        for col in range(3):
            x, y = 220 + col * 260, 80 + row * 200
            draw.rectangle([x, y, x + 240, y + 180], fill='#1a2535', outline='#2a3a4f')
            draw.text((x + 20, y + 20), f"Metric {row * 3 + col + 1}", fill='#808080')
            draw.text((x + 20, y + 60), f"${(row * 3 + col + 1) * 1234:,}", fill='#00d084')
    
    # Buttons
    for i, (text, color) in enumerate([("Export", '#3a7bd5'), ("Refresh", '#00a86b')]):
        x = 220 + i * 150
        draw.rectangle([x, height - 80, x + 130, height - 40], fill=color)
        draw.text((x + 20, height - 70), text, fill='#ffffff')
    
    return img

test_image = create_test_image()
print(f"📸 Created test image: {test_image.size}")

# Display thumbnail
thumb = test_image.copy()
thumb.thumbnail((600, 600))
display(thumb)


In [ ]:
# Cell 5: Prompt Format Function
def format_prompt(instruction: str, model_name: str) -> str:
    """Format prompt based on model family."""
    system = "You are a helpful visual assistant. Analyze images carefully."
    
    return (
        f"<|im_start|>system\n{system}<|im_end|>\n"
        f"<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>{instruction}<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

print("✅ Prompt functions defined")


In [ ]:
# Cell 6: Run Single Image Inference
sampling_params = SamplingParams(temperature=0.0, max_tokens=512, stop=["<|im_end|>"])

instructions = [
    "Describe this dashboard interface.",
    "What menu items are visible in the sidebar?",
    "What is the total value of all metrics shown?",
]

print("⚡ Running inference...")
print("=" * 80)

model_name = config["model"]
results = []

for i, instruction in enumerate(instructions, 1):
    prompt = format_prompt(instruction, model_name)
    inputs = {"prompt": prompt, "multi_modal_data": {"image": test_image}}
    
    start = time.time()
    outputs = llm.generate([inputs], sampling_params=sampling_params)
    latency = (time.time() - start) * 1000
    
    response = outputs[0].outputs[0].text
    tokens = len(outputs[0].outputs[0].token_ids)
    
    print(f"\n[{i}] {instruction}")
    print(f"    ⏱️  Latency: {latency:.0f}ms | Tokens: {tokens} | Speed: {tokens/(latency/1000):.1f} tok/s")
    print(f"    📝 Response: {response[:200]}..." if len(response) > 200 else f"    📝 Response: {response}")
    
    results.append({"instruction": instruction, "latency": latency, "tokens": tokens})

avg_latency = sum(r["latency"] for r in results) / len(results)
print(f"\n📊 Average latency: {avg_latency:.0f}ms")


In [ ]:
# Cell 7: Batch Inference (High Throughput)
batch_instructions = [
    "Click on Dashboard", "Click on Analytics", "Click on Reports",
    "Click on Settings", "Click on Users", "Click on Export",
    "Click on Refresh", "Click on Metric 1",
]

batch_inputs = [
    {"prompt": format_prompt(inst, model_name), "multi_modal_data": {"image": test_image}}
    for inst in batch_instructions
]

print(f"⚡ Batch inference: {len(batch_inputs)} requests")

batch_start = time.time()
batch_outputs = llm.generate(batch_inputs, sampling_params=sampling_params)
batch_time = (time.time() - batch_start) * 1000

print(f"\n✅ Completed in {batch_time:.0f}ms")
print(f"📈 Throughput: {len(batch_inputs) / (batch_time / 1000):.1f} requests/second")
print(f"📈 Per-request: {batch_time / len(batch_inputs):.0f}ms")


In [ ]:
# Cell 9: Complete Architecture & GPU Summary
print("""
╔═══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                        COMPLETE QWEN VL ARCHITECTURE COMPARISON                                   ║
╠═══════════════════════════════════════════════════════════════════════════════════════════════════╣
║                                                                                                   ║
║  ┌────────────────────────┬─────────────────────────────┬─────────────────────────────────────┐   ║
║  │ Component              │ Qwen2-VL                    │ Qwen3-VL                            │   ║
║  ├────────────────────────┼─────────────────────────────┼─────────────────────────────────────┤   ║
║  │ VISION ENCODER                                                                             │   ║
║  │ ─────────────────────────────────────────────────────────────────────────────────────────  │   ║
║  │ Patch Embedding        │ Conv3D bias=False           │ Conv3D bias=True                    │   ║
║  │ Position Encoding      │ 3D RoPE only                │ Learned + Bilinear Interp + RoPE    │   ║
║  │ MLP Activation         │ QuickGELU: x*σ(1.702x)      │ SiLU: x*σ(x)                        │   ║
║  │ MLP Bias               │ Has bias                    │ No bias (linear_fc1/fc2)            │   ║
║  │ Feature Extraction     │ Final layer only            │ Multi-scale (DeepStack)             │   ║
║  │ Video Max Frames       │ 14 frames                   │ 24,576 frames                       │   ║
║  ├────────────────────────┼─────────────────────────────┼─────────────────────────────────────┤   ║
║  │ LLM INTEGRATION                                                                            │   ║
║  │ ─────────────────────────────────────────────────────────────────────────────────────────  │   ║
║  │ Backbone               │ Qwen2ForCausalLM            │ Qwen3ForCausalLM                    │   ║
║  │ Visual Injection       │ Embedding level only        │ Embed + Early layer injection       │   ║
║  │ DeepStack              │ ❌ Not supported            │ ✅ Multi-scale feature injection    │   ║
║  │ EVS (Video Pruning)    │ ❌ Not supported            │ ✅ 30-70% token reduction           │   ║
║  ├────────────────────────┼─────────────────────────────┼─────────────────────────────────────┤   ║
║  │ OPTIMIZATION                                                                               │   ║
║  │ ─────────────────────────────────────────────────────────────────────────────────────────  │   ║
║  │ Attention Backends     │ FlashAttn, SDPA, Xformers   │ FlashAttn, SDPA only                │   ║
║  │ Torch Compile          │ Limited                     │ @support_torch_compile              │   ║
║  │ Eagle3 Speculative     │ ❌ Not supported            │ ✅ SupportsEagle3 interface         │   ║
║  │ Multimodal Pruning     │ ❌ Not supported            │ ✅ SupportsMultiModalPruning        │   ║
║  └────────────────────────┴─────────────────────────────┴─────────────────────────────────────┘   ║
║                                                                                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════════╝

╔═══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                              GPU-SPECIFIC OPTIMIZATION GUIDE                                      ║
╠═══════════════════════════════════════════════════════════════════════════════════════════════════╣
║                                                                                                   ║
║  T4 (TURING SM 7.5, 16GB GDDR6, 320 GB/s)                                                         ║
║  ═════════════════════════════════════════                                                        ║
║  Constraints: No FlashAttn v2, No BF16, No FP8, Limited memory                                    ║
║  Recommended: enforce_eager=True, max_num_seqs=4, max_pixels=512000                               ║
║  Best models: Qwen2-VL-2B (FP16), Qwen3-VL-4B (4-bit with EVS)                                    ║
║  Expected: ~800-1200ms latency, ~15-25 tok/s                                                      ║
║                                                                                                   ║
║  A100 (AMPERE SM 8.0, 80GB HBM2e, 2.0 TB/s)                                                       ║
║  ═════════════════════════════════════════                                                        ║
║  Features: FlashAttn v2, BF16, Tensor Cores                                                       ║
║  Recommended: dtype=bfloat16, max_num_seqs=32, enable_prefix_caching=True                         ║
║  Best models: Qwen2-VL-7B, Qwen3-VL-8B, Qwen3-VL-30B-A3B (MoE)                                    ║
║  Expected: ~200-400ms latency, ~80-120 tok/s                                                      ║
║                                                                                                   ║
║  H100 (HOPPER SM 9.0, 80GB HBM3, 3.35 TB/s)                                                       ║
║  ═════════════════════════════════════════                                                        ║
║  Features: FlashAttn v3, FP8, Transformer Engine, 10x T4 bandwidth                                ║
║  Recommended: FP8 quant for max throughput, kv_cache_dtype=fp8                                    ║
║  Best models: Qwen3-VL-8B (32K ctx), Qwen3-VL-8B FP8 (64K ctx!)                                   ║
║  Expected: ~100-200ms latency, ~120-180 tok/s                                                     ║
║                                                                                                   ║
║  B200 (BLACKWELL SM 10.0, 192GB HBM3e, 8.0 TB/s)                                                  ║
║  ═════════════════════════════════════════                                                        ║
║  Features: FP4 (future), 25x T4 bandwidth, massive memory                                         ║
║  Recommended: max_model_len=131072 (128K!), max_pixels=4147200 (4K)                               ║
║  Best models: Qwen3-VL-8B (128K ctx), Qwen2-VL-72B (single GPU!)                                  ║
║  Expected: ~50-100ms latency, ~180-300 tok/s                                                      ║
║                                                                                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════════╝

╔═══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                  DEEPSTACK EXPLAINED                                              ║
╠═══════════════════════════════════════════════════════════════════════════════════════════════════╣
║                                                                                                   ║
║  DeepStack extracts features from INTERMEDIATE vision layers and injects them into EARLY          ║
║  language model layers. This provides multi-resolution visual information to the LLM.             ║
║                                                                                                   ║
║  Vision Encoder:                                                                                  ║
║    Layer 0  ───────────────────────────────────────────────────────────────────────►              ║
║    ...                                                                                            ║
║    Layer k  ──────────► DeepStack Merger [0] ──────────────────┐                                  ║
║    ...                                                          │                                  ║
║    Layer m  ──────────► DeepStack Merger [1] ──────────────────┼─┐                                ║
║    ...                                                          │ │                                ║
║    Layer N  ──────────► Main Merger ────────────────────────┐  │ │                                ║
║                                                              │  │ │                                ║
║                                  CONCATENATE: [main | ds0 | ds1]                                  ║
║                                              ▼                                                     ║
║  LLM Backbone:                                                                                    ║
║    Layer 0: hidden += deepstack_embeds[0]  ◄── Early injection                                    ║
║    Layer 1: hidden += deepstack_embeds[1]  ◄── Early injection                                    ║
║    Layer 2: normal forward                                                                        ║
║    ...                                                                                            ║
║                                                                                                   ║
║  BENEFIT: LLM has access to low-level (edges), mid-level (shapes), and high-level (semantics)    ║
║           visual features throughout its processing, not just at the input embedding level.       ║
║                                                                                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════════╝

╔═══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                            EVS (EFFICIENT VIDEO SAMPLING) EXPLAINED                               ║
╠═══════════════════════════════════════════════════════════════════════════════════════════════════╣
║                                                                                                   ║
║  EVS uses content-aware pruning based on frame-to-frame similarity:                               ║
║                                                                                                   ║
║  Algorithm:                                                                                       ║
║    1. Compute cosine similarity between consecutive frames                                        ║
║    2. dissimilarity = 1 - similarity (higher = more different = more important)                  ║
║    3. First frame always kept (dissimilarity set to max)                                          ║
║    4. Sort all tokens by dissimilarity, keep top (1-q) fraction                                   ║
║    5. Recompute M-RoPE positions for retained tokens                                              ║
║                                                                                                   ║
║  Example (60s video, 120 frames):                                                                 ║
║    Without EVS: ~19,200 tokens → may exceed context window!                                       ║
║    With EVS (q=0.5): ~9,600 tokens → 2x faster, fits in context                                   ║
║                                                                                                   ║
║  Configuration:                                                                                   ║
║    mm_processor_kwargs = {"video_pruning_rate": 0.5}  # Prune 50%                                 ║
║                                                                                                   ║
║  Quality vs Speed tradeoff:                                                                       ║
║    q=0.3 (30% pruning): Minimal quality loss, 1.4x speedup                                        ║
║    q=0.5 (50% pruning): Slight quality loss, 2x speedup                                           ║
║    q=0.7 (70% pruning): Noticeable loss, 3x speedup                                               ║
║                                                                                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════════╝
""")

print("✅ Notebook complete!")
print("\\n📚 For more details, see: qwen_vl_deep_dive.md")
